In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool, cpu_count
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [ ]:
SNIPPET_DIR_PATH = '/home/unnc/Documents/_data/_snippet/'
ORIGIN_DIR_PATH = '/home/unnc/Documents/_data/_original_data/'


TRAIN_NAME = f'train'

TEST_17_NAME = f'test_17'
TEST_18_NAME = f'test_18'
TRAIN_3000_NAME = f'train_3000'

In [ ]:
TRUNC_OUTPUT_DIR_PATH = '/home/unnc/Desktop/trunc/' # QC1 QC2 QC3 QC4 QC5
# SWAG_OUTPUT_DIR_PATH = '/home/unnc/Documents/_data/swag/' # Q C1 C2 C3 C4 C5


In [ ]:
# original data
train_original_data = pd.read_excel(f'{ORIGIN_DIR_PATH}train.xlsx', header=None)
test_original_data_17 = pd.read_excel(f'{ORIGIN_DIR_PATH}test_17-18.xlsx', sheet_name='2017', header=None)
test_original_data_18 = pd.read_excel(f'{ORIGIN_DIR_PATH}test_17-18.xlsx', sheet_name='2018', header=None)

## Level 5_3

In [ ]:
snippet_type = '5_3'
output_dir = f'lvl_{snippet_type}'

In [ ]:
# snippet 
snippet_train = pd.read_excel(f'{SNIPPET_DIR_PATH}/训练集查询结果/带答案搜索/level{snippet_type}.xlsx', header=None)
snippet_test_17 = pd.read_excel(f'{SNIPPET_DIR_PATH}/2017真题查询结果/带答案搜索/indexLevels{snippet_type}.xlsx', header=None)
snippet_test_18 = pd.read_excel(f'{SNIPPET_DIR_PATH}/2018真题查询结果/带答案搜索/indexLevels{snippet_type}.xlsx', header=None)

In [ ]:
# concatenate orginal data with snippet
train_with_snippet = pd.concat([train_original_data, snippet_train], axis=1)
test_17_all = pd.concat([test_original_data_17, snippet_test_17], axis=1)
test_18_all = pd.concat([test_original_data_18, snippet_test_18], axis=1)

# Data cleaning

### drop corruped data

In [ ]:
# 1. handle corrupted row at 10040
if train_with_snippet[train_with_snippet.iloc[:,8].isnull() != True].iloc[:,0].tolist() != []:
    print(train_with_snippet[train_with_snippet.iloc[:,8].isnull() != True].iloc[:,0].tolist())
    train_with_snippet.drop(train_with_snippet.index[10040], inplace=True)

In [ ]:
# 2. handle wrong choice (longer than 400) at 21511
if len(train_with_snippet.iloc[21509,2]) > 400:
    print(train_with_snippet.iloc[21509,2])
    train_with_snippet.drop(train_with_snippet.index[21509], inplace=True)
    print('dropped')

### drop nan entries

In [ ]:
# 0: question
# 1: question type
# 2-6: choices
# 7: answer
# 10: textbook snippet
# 11-14: title level 1-4
train_data = train_with_snippet.iloc[:,[0,1,2,3,4,5,6,7,10,11,12,13,14]]

In [ ]:
train_with_snippet.head(1)

In [ ]:
# 0: question
# 1: question type
# 2-6: choices
# 7: answer
# 8: textbook snippet: best
# 9-14: title level 1-4
test_17_data = test_17_all.iloc[:,[0,1,2,3,4,5,6,7, 8, 9,10,11,12,13]]
test_18_data = test_18_all.iloc[:,[0,1,2,3,4,5,6,7, 8, 9,10,11,12,13]]

In [ ]:
test_17_data.head(1)

In [ ]:
# drop entries that has any nan and, 
# print num of nan in each col
def any_nan_values(df):
    return df.isnull().values.any()

def drop_nan(df):
    if any_nan_values(df):
        print("nan in each col:\n", df.isnull().sum(), sep='')
        
        return df.dropna()
    return df

In [ ]:
train_data = drop_nan(train_data)
# reset index after dropping rows
train_data.reset_index(inplace=True)
train_data = train_data.drop(columns='index')

In [ ]:
# assert every cell is not nan
assert not any_nan_values(train_data) and not any_nan_values(test_17_data) and not any_nan_values(test_18_data)

In [ ]:
train_data.shape

### add column headers

In [ ]:
train_data.columns = ['q', 'q_type', 'c1', 'c2', 'c3', 'c4', 'c5', 'a', 's1', 's1t1', 's1t2', 's1t3', 's1t4']

test_17_data.columns = ['q','c1','c2','c3','c4','c5', 'q_type', 'year', 'a', 's1','s1t1','s1t2','s1t3','s1t4']
test_18_data.columns = ['q','c1','c2','c3','c4','c5', 'q_type', 'year', 'a', 's1','s1t1','s1t2','s1t3','s1t4']

In [ ]:
train_data.head(1)

In [ ]:
# rearange df columns to match the ones in training data
test_17_data = test_17_data[train_data.columns]
test_18_data = test_18_data[train_data.columns]

In [ ]:
# Draw question len && snippet len graph 
def draw_q_s_len(df):
    q_len = [len(str(i).replace(' ', '')) for i in df['q']]
    s1_len = [len(str(i).replace(' ', '')) for i in df['s1']]

    len_512 = round(sum([i > 512 for i in s1_len]) / len(s1_len), 2)
    len_756 = round(sum([i > 756 for i in s1_len]) / len(s1_len), 2)
    len_1024 = round(sum([i > 1024 for i in s1_len]) / len(s1_len), 2)

    label = f'snippet:   >512: {len_512}       >756: {len_756}      >1024: {len_1024}'

    a = sns.distplot(s1_len, kde=False, axlabel=label)
    a = sns.distplot(q_len, kde=False)
    a = a.get_figure()
    a.savefig(f'/{TRUNC_OUTPUT_DIR_PATH}/{output_dir}/qs.png', dpi=120, bbox_inches='tight')
    print('saved to:', TRUNC_OUTPUT_DIR_PATH)

In [ ]:
draw_q_s_len(train_data)

# Add textbook snippit - only the best match

In [ ]:
def combine_titles(df_titles):
    """
    df_titles: any cleaned df that contains title information 
    
    return: list
    """
    return [' '.join(titles) for titles in df_titles[['s1t1', 's1t2', 's1t3', 's1t4']].values]


def combine_qst(df_data, add_q=True, add_s=True, add_t=True):
    """
    return: list
    
    output sequence is [title + question + snippet]
    """
    result = []
    if add_t:
        t = combine_titles(df_data)
    q = df_data['q'].values
    s1 = df_data['s1'].values
    
    for i in range(df_data.shape[0]):
        assert 'æ' not in t[i] and 'æ' not in q[i] and 'æ' not in s1[i], 'Input data contains æ sign'
        all_tqs = [t[i], q[i], s1[i]] # change output sequence here
        input_tqs = [] 
        for i, add in enumerate([add_t, add_q, add_s]): # should be the same sequence as all_tqs
            if add:
                input_tqs.append(all_tqs[i])
        input_row = '  æ  '.join(input_tqs)
        result.append(input_row)
    print(f'First output is \n{result[0]}')
    return result

In [ ]:
def get_trunc_df(df_data):
    qst_list = combine_qst(df_data, True, True, True)
    df_trunc = df_data[['q','q_type','c1','c2','c3','c4','c5','a']].copy()
    df_trunc['q'] = qst_list
    return df_trunc
def add_question_index(df_data):
    question_len = df_data.shape[0]
    df_data['q_index'] = range(question_len)
    return df_data

In [ ]:
train_trunc = get_trunc_df(train_data)
train_trunc = add_question_index(train_trunc)

In [ ]:
test_17_trunc = get_trunc_df(test_17_data)
test_17_trunc = add_question_index(test_17_trunc)

In [ ]:
test_18_trunc = get_trunc_df(test_18_data)
test_18_trunc = add_question_index(test_18_trunc)
assert test_18_trunc.shape[0] == 599 and test_17_trunc.shape[0] == 600

In [ ]:
def draw_input_len(df):
    input_len = [len(i.replace(' ', '')) for i in df['q']]
    len_512 = round(sum([i > 512 for i in input_len]) / len(input_len), 2)
    len_756 = round(sum([i > 756 for i in input_len]) / len(input_len), 2)
    len_1024 = round(sum([i > 1024 for i in input_len]) / len(input_len), 2)

    label = f'>512: {len_512}       >756: {len_756}      >1024: {len_1024}'
    a = sns.distplot(input_len, kde=False, axlabel=label)
    a = a.get_figure()
    a.savefig(f'/{TRUNC_OUTPUT_DIR_PATH}/{output_dir}/input_len.png', dpi=120, bbox_inches='tight')

In [ ]:
draw_input_len(train_trunc)

In [ ]:
def shuffle_df(df, frac=1, random_state=42):
    return df.sample(frac=frac, random_state=random_state)

In [ ]:
train_trunc_shuffled = shuffle_df(train_trunc)

In [ ]:
train_trunc_shuffled.head(1)

# 1. TRUNC method

For a single question

$QC_1...C_5 \longrightarrow \sum_{c=1}^{5} QC_c$

question + 5 choice -> 5 * (question + 1 choice)

In [ ]:
def expand_choices(df):
    """
    return:
    df with expanded choice
    """
    
    result = []
    question_index = df.columns.get_loc('q')
    first_choice_index = df.columns.get_loc('c1')
    answer_index = df.columns.get_loc('a')
    # iterate through all entries in df
    for i in tqdm(range(df.shape[0])):
        one_entry = df.iloc[i,:] 
        # for each entry, take its 5 choices in sequence into 5 [question, one_choice, label] outputs
        for choice_index in range(5):
            label = 1 if (choice_index + 1)  == one_entry[answer_index] else 0
            result.append({'input': one_entry[question_index], 
                       'choice': one_entry[first_choice_index+choice_index],
                       'q_type': one_entry['q_type'],
                       'c_index': choice_index, 
                       'q_index': one_entry['q_index'],
                       'label': label})
    return pd.DataFrame(result)[['input','q_index','choice','c_index','label','q_type']]

In [ ]:
train_trunc_df = expand_choices(train_trunc_shuffled)

In [ ]:
test_17_trunc_df = expand_choices(test_17_trunc)

In [ ]:
test_18_trunc_df = expand_choices(test_18_trunc)

### Save files to help calculating accuracy

In [ ]:
def save_debug_file(df, file_name):
    df.to_excel(f'{TRUNC_OUTPUT_DIR_PATH}/{output_dir}/{file_name}_debug.xlsx', index=None)
    print(f'{file_name} saved!')

In [ ]:
save_debug_file(test_17_trunc_df, TEST_17_NAME)
save_debug_file(test_18_trunc_df, TEST_18_NAME)
save_debug_file(train_trunc_df.head(15000), TRAIN_3000_NAME)

### Preparing for bert

In [ ]:
def prepare_for_bert(df):
    return pd.DataFrame({
        'id':range(df.shape[0]),
        'label':df['label'],
        'alpha':['a']*df.shape[0],
        'text_a': df['input'].replace(r'\n', ' ', regex=True),
        'text_b': df['choice'].replace(r'\n', ' ', regex=True)
    })

In [ ]:
train_trunc_bert = prepare_for_bert(train_trunc_df)

test_17_trunc_bert = prepare_for_bert(test_17_trunc_df)
test_18_trunc_bert = prepare_for_bert(test_18_trunc_df)
train_3000_trunc_bert = prepare_for_bert(train_trunc_df.head(15000))

In [ ]:
def save_input_file(df_bert, file_name):
    df_bert.to_csv(f'{TRUNC_OUTPUT_DIR_PATH}/{output_dir}/{file_name}.tsv', 
                sep='\t', index=False, header=False)
    print(f'{file_name} saved!')

In [ ]:
save_input_file(train_trunc_bert, TRAIN_NAME)

In [ ]:
save_input_file(test_17_trunc_bert, TEST_17_NAME)
save_input_file(test_18_trunc_bert, TEST_18_NAME)
save_input_file(train_3000_trunc_bert, TRAIN_3000_NAME)